In [ ]:
# This has to go in its own cell or it screws up the defaults we'll set later
%matplotlib inline

### Music visualization for image classification

Music classification and similarity detection seems like an obvious application of machine learning, but it's a hard problem and there is still [a lot of research work going on](https://magenta.tensorflow.org/). With image analysis, by contrast, there are lots of well-understood networks, some of them [included with Keras](https://keras.io/applications/), which means that you can set up an image classifier in about a dozen lines of Python. Here's a simple demo I put together in 2017, in fact, that lets you [try them all out using your webcam](https://github.com/plaidml/plaidvision). 

Could take advantage of this by generating pictures of audio waveforms and training an image classificaton network to identify them? Let's try out some different techniques for rendering music in a format suitable for an image classifier and see if we can find a representation that makes sense to humans. If we can see the patterns in the images we've generated, and we can readily distinguish the differences between images generated for different pieces of music, it seems probable that a robot might be able to learn to do the same.

#### Setting up

We'll be using Python for this project, with [NumPy](https://pypi.org/project/numpy/), [matplotlib](https://pypi.org/project/matplotlib/), and [LibROSA](https://pypi.org/project/librosa/).

In [ ]:
import numpy as np
import librosa, librosa.display
import matplotlib
from matplotlib import pyplot

In [ ]:
matplotlib.rcParams['figure.figsize']=(15.0, 3.0)
matplotlib.rcParams['image.cmap']='magma'
pyplot.rcParams['image.cmap']='magma'

We can examine a variety of other music later, but for now let's just use the example file that comes with librosa. An audio file is basically just one long array of amplitude samples and a "sampling rate" parameter specifying the number of samples per second.

In [ ]:
#file_path = "/home/mars/Music/Track Library/04 Groove And Move.mp3"
file_path = "/home/mars/Music/Track Library/Tribone & Whitebear - Pure Self.mp3"
signal, sr = librosa.load(file_path)
print "The audio data is a %s and its length is %d." % (type(signal), len(signal))

#### Plotting audio features

Let's begin with the most straightforward approach by simply plotting the amplitudes, producing a classic waveform image like many you've seen before:

In [ ]:
librosa.display.waveplot(signal);

This view tells us something about the structure of the music, but it's not very detailed. All we see are the limits of the waveforms. What's going on inside? Let's get a more detailed look by constructing an amplitude histogram. We'll slice the signal into frames, divide the amplitude range into bins, and count the number of samples in each frame which land in that bin. We can plot the resulting 2D array by applying a color map, so the brightness or darkness of each point is proportional to the number of values in the bin.

In [ ]:
frames = np.array_split(signal, 1000)
amplitude_histogram = np.array([bins for bins, edges in (np.histogram(f, bins=100) for f in frames)])
# Convert to float and transpose from [frames, bins] to [bins, frames] order.
amplitude_histogram = amplitude_histogram.astype(np.float).transpose()
# Normalize by dividing each value against the maximum in its column.
amplitude_histogram /= amplitude_histogram.max(axis=0)
librosa.display.specshow(amplitude_histogram, cmap='Blues');

This certainly brings out more of the structure; we can clearly see the difference between a generally quiet section which contains some loud peaks and a section which is loud overall, and we can make some guesses about rhythm patterns. We still don't know very much about the character of the sound, though: we know nothing about its pitches or timbres.

 We can extract the frequency content from the signal using an algorithm called the "short-time Fourier transform", or STFT. This process splits the array of samples into evenly-sized frames, computes the Fourier transform on each frame, and returns a two-dimensional array of frequency components by frames. 

In [ ]:
n_fft = 2048
spectrogram = librosa.stft(signal, n_fft=n_fft)
print "The spectrogram contains %d frequency components for each of %d frames." % spectrogram.shape
print "The array value type is '%s'." % spectrogram.dtype

The Fourier transform returns complex numbers including the magnitude and phase for each frequency component. This is more information than we need, so we'll extract the magnitude and throw away the phase by taking the absolute value, allowing us to simplify things by using ordinary floats instead of complex numbers.

In [ ]:
spectrogram = np.abs(spectrogram).astype(np.float)

Let's get a look at this spectrogram. Since it's a two-dimensional array, we'll draw frames horizontally and frequency components vertically, representing magnitude with a color map from dark to bright.

In [ ]:
librosa.display.specshow(spectrogram, cmap='magma');

All we see is a few dots along the bottom - that can't be right! What happened?

Human senses are logarithmic, not linear, so we perceive small differences between small numbers to be about the same as large differences between large numbers. The levels of different frequencies in an audio signal may be a dozen orders of magnitude apart. We can clearly distinguish subtle changes in quiet frequencies while also hearing the large changes in the dominant frequencies, but when we try to plot those amplitudes on a linear scale, the loudest bands drown everything else out.

Let's try to get a better model of perception by putting these amplitudes on a log scale, converting them to decibels.

In [ ]:
db_spectrogram = librosa.amplitude_to_db(spectrogram)
librosa.display.specshow(db_spectrogram, cmap='magma');

We're getting closer, but this still looks pretty strange. It's not just our sense of volume that is logarithmic, but our sense of pitch, too. We are far more sensitive to the the differences in lower frequencies than higher ones. One common logarithmic model of pitch sensitivity is called the Mel scale. Let's use a mel-scaled filterbank to condense our spectrogram down to a smaller array of more useful numbers.

In [ ]:
# Create a filterbank which will convert an array [n_fft,frames] to [n_mels,frames].
# This filterbank will be an array of weights [n_mels, n_fft].
melfilterbank = librosa.filters.mel(sr, n_fft=n_fft, n_mels=24, fmin=50.0, fmax=8000.0)
mel_spectrogram = melfilterbank.dot(spectrogram)
db_mel_spectrogram = librosa.amplitude_to_db(mel_spectrogram)
librosa.display.specshow(db_mel_spectrogram, cmap='magma');

Now we can see a difference between activity in low, middle, and high frequencies, and we can even get some sense of melodic variations and other musical events.

In [ ]:
# Compute perceptual weighting on the mel-scaled power spectrogram
mel_frequencies = librosa.mel_frequencies(n_mels=24, fmin=50.0, fmax=8000.0)
perceptual_melspec = librosa.perceptual_weighting(mel_spectrogram**2, mel_frequencies, ref=np.max)
librosa.display.specshow(perceptual_melspec, cmap='magma');

#### Creating images for object detection

Armed with these basic techniques for turning sounds into images, let's consider the problem of preparing input images for an image classification network.

Each network is designed for a specific input size, which is usually no more than 224x224 pixels - Xception goes up to 299x299 pixels. That creates some challenges for us; it's not a lot of space, and the square aspect ratio doesn't really suit the horizontal, timeline-based approaches we've been using. Let's see what happens if we simply squash our existing plots into shape:

In [ ]:
pyplot.figure(dpi=100, figsize=(9,3))
pyplot.subplot(131).imshow(amplitude_histogram, extent=[0,224,0,224], cmap='Blues')
pyplot.subplot(132).imshow(db_mel_spectrogram, extent=[0,224,0,224], cmap='magma', origin='lower')
pyplot.subplot(133).imshow(perceptual_melspec, extent=[0,224,0,224], cmap='magma', origin='lower')
pyplot.show()

This is a pretty thumbnail image, but the horizontal compression has sacrificed so much time resolution that there's  not enough information here to recognize a track, much less to guess what kind of music it might be, or to estimate the similarity between two different pieces of music.

What's more, music is full of repeating patterns and variations on patterns, which all disappear in this view. The timeline view places these repetitions apart in time, even though they could be considered very close in terms of musical similarity. If we are interested in training a classifier network to recognize musical similarity, we should find some way to make the hierarchy of rhythmic structure more prominent.

In [ ]:
#### Tempo synchronization

Let's see what happens if we try to synchronize our display with tempo. If we're trying to create an image that has 224x224 pixels, we could use 

In [ ]:
tempo = librosa.beat.tempo(signal, sr=sr)[0]
print tempo

In [ ]:
tempo = 128.0
seconds_per_beat = 60.0 / tempo
samples_per_beat = seconds_per_beat * float(sr)
total_beats = len(signal) / samples_per_beat
print "samples_per_beat == %.2f, total_beats == %.2f" % (samples_per_beat, total_beats)

In [ ]:
def centerize_spectrogram(spec):
    out = np.zeros_like(spec)
    midpoint = int(spec.shape[0] / 2)
    out[midpoint:,...] = spec[0::2,...]
    out[midpoint-1::-1,...] = spec[1::2,...]
    return out

In [ ]:
def render_wrapped(number_of_rows):
    # Compute the number of beats which is the nearest power of 2.
    beats_per_row = 2 ** int(np.floor(  np.log(total_beats / number_of_rows) / np.log(2.0)  ))
    beats_to_use = beats_per_row * number_of_rows

    # set the hop length so that each row will divide evenly into 224 slices
    samples_per_row = int(beats_per_row * samples_per_beat)
    hop_length = samples_per_row / 224
    while hop_length >= 2048:
        hop_length /= 2
        
    samples_to_use = int(np.floor(beats_to_use * samples_per_beat))
    sample_start = int((len(signal) - samples_to_use) / 2)
    sample_end = sample_start + samples_to_use
    display_clip = signal[sample_start:sample_end]

    pixels_per_row = 224 / number_of_rows
    mel_spec = librosa.feature.melspectrogram(
        y=display_clip, sr=sr, n_mels=pixels_per_row, fmin=50.0, fmax=8000, hop_length=hop_length)
    mel_frequencies = librosa.mel_frequencies(n_mels=pixels_per_row, fmin=50.0, fmax=8000.0)
    display_spec = librosa.perceptual_weighting(mel_spec**2, mel_frequencies, ref=np.max)

    frames_per_row = int(np.floor(display_spec.shape[1] / number_of_rows))
    row_specs = [display_spec[:,i*frames_per_row:(i+1)*frames_per_row] for i in range(number_of_rows)]
    row_specs = [centerize_spectrogram(s) for s in row_specs]
    row_specs = np.concatenate(row_specs, axis=0)
    
    # if we have surplus columns, average them together
    while row_specs.shape[1] > 224:
        print "condensing columns from %d to %d" % (row_specs.shape[1], row_specs.shape[1] / 2)
        row_specs = (row_specs[:,0:-1:2] + row_specs[:,1::2]) / 2.0
        
    print "number_of_rows == %d, row_specs == %s" % (number_of_rows, row_specs.shape)
    return row_specs


In [ ]:
# We would like rows and columns to be proportional.
# The number of columns must be a power of 2.
# The number of rows must divide evenly into 224.
estimated_dimension = np.sqrt(total_beats)
# Round down to the nearest integer which is a factor of 224
# and no smaller than 7.
pixels_per_row = max(7, int(np.ceil(224.0 / estimated_dimension)))
number_of_rows = int(np.floor(224.0 / pixels_per_row))

tiled_image = render_wrapped(number_of_rows)

pyplot.figure(figsize=(6,6))
pyplot.imshow(tiled_image, cmap='magma');

#### mapping linear time through planar space
wrapping rows
quadtree decomposition
the hilbert curve
self-similarity matrix

tempo synchronization: is it a win?

#### three dimensions of depth: color
different color spaces: RGB, HSV, HSL, Lab
different audio features: magnitude, power; spectral centroid, spread, rolloff, flatness

In [ ]:
def tiled_spectrogram(num_tiles, tile_size):
    hop_length = int(len(signal) / (num_tiles * tile_size))
    merge_factor = 1
    while hop_length >= 2048:
        hop_length /= 2
        merge_factor *= 2
    # compute a perceptually-scaled spectrogram with the specified hop length
    mel_spec = librosa.feature.melspectrogram(
        y=signal, sr=sr, n_mels=tile_size, fmin=50.0, fmax=8000, hop_length=hop_length)
    mel_frequencies = librosa.mel_frequencies(n_mels=tile_size, fmin=50.0, fmax=8000.0)
    display_spec = librosa.perceptual_weighting(mel_spec**2, mel_frequencies, ref=np.max)
    display_spec = centerize_spectrogram(display_spec)
    # average out adjacent frames until it has the right length
    while merge_factor > 1:
        display_spec = (display_spec[:,0:-1:2] + display_spec[:,1::2]) / 2.0
        merge_factor /= 2
    return np.array_split(display_spec[:,:num_tiles*tile_size], num_tiles, axis=1)


In [ ]:
def spacefill(matrix):
    x, y = 1, 1
    while x < matrix.shape[1] or y < matrix.shape[0]:
        nx = min(matrix.shape[1], x*2)
        if nx > x:
            clipwidth = nx - x
            source = matrix[0:y, 0:clipwidth]
            matrix[0:y, x:nx] = source + source.size
            x = nx

        ny = min(matrix.shape[0], y*2)
        if ny > y:
            clipheight = ny - y
            source = matrix[0:clipheight, 0:x]
            matrix[y:ny, 0:x] = source + source.size
            y = ny


In [ ]:
# this matrix is an indexer from 0 through the number of items it contains.
# we'll divide it by its number of items to get a 0..1 range, then scale it
# up to the number of frames in our spectrogram, so that each cell will
# select one frame.

matrix = np.zeros(shape=(28,28), dtype=np.int)
spacefill(matrix)

number_of_cells = matrix.size
cell_dimension = 224 / 28
spec_tiles = tiled_spectrogram(number_of_cells, cell_dimension)
reshaped_spec = np.asarray(spec_tiles)[matrix.reshape(-1),:,:].reshape(224,224)
pyplot.figure(figsize=(6,6))
pyplot.imshow(reshaped_spec, cmap='magma');

In [ ]:
def hilbert_path(power):
    path = [(0, 0)]
    for i in range(power):
        s = 1 + (path[-1][0] - path[0][0]) / 2
        path = ( [(y-s, x-s) for x,y in path]
               + [(x-s, y+s) for x,y in path]
               + [(x+s, y+s) for x,y in path]
               + [(s-y,-x-s) for x,y in path] )
    return path

def hilbert_matrix(power):
    size = 2 ** power
    matrix = np.zeros((size, size), dtype=np.int)
    for i, (x, y) in enumerate(hilbert_path(power)):
        x = (x - 1 + size) / 2
        y = (y - 1 + size) / 2
        matrix[x,y] = i
    return matrix
    
#pyplot.figure(figsize=(8,8))
#pts = hilbert_path(4)
#pyplot.plot(*zip(*pts))
#pyplot.show()

number_of_cells = 16 * 16
cell_dimension = 224 / 16
spec_tiles = tiled_spectrogram(number_of_cells, cell_dimension)

matrix = hilbert_matrix(4)
print matrix
reshaped_spec = np.asarray(spec_tiles)[matrix.reshape(-1),:,:].reshape(224,224)
pyplot.figure(figsize=(6,6))
pyplot.imshow(reshaped_spec, cmap='magma');

In [ ]:
def hsl_to_rgb(h, s, l):
    shape = h.shape
    
    # all inputs and outputs range 0..1
    r = np.zeros(shape)
    g = np.zeros(shape)
    b = np.zeros(shape)
    
    # where the color is totally desaturated, use only luminance
    grey = (s==0)
    r[grey] = l[grey]
    g[grey] = l[grey]
    b[grey] = l[grey]
    
    # scale the saturation differently around medium luminance
    q = np.zeros(shape)
    low_luma = l < 0.5
    q[low_luma] = l[low_luma] * (1 + s[low_luma])
    hi_luma = l >= 0.5
    q[hi_luma] = l[hi_luma] + s[hi_luma] - l[hi_luma] * s[hi_luma]
    # the other hue factor is proportional
    p = 2 * l - q;
    
    def channel(t):
        # enforce limits
        t[t < 0] += 1.0
        t[t > 1] -= 1.0
        x = np.zeros(shape)
        tA = t < 1/6.
        x[tA] = p[tA] + (q[tA] - p[tA]) * 6 * t[tA] 
        tB = (t >= 1/6.) & (t < 1/2.)
        x[tB] = q[tB]
        tC = (t >= 1/2.) & (t < 2/3.)
        x[tC] = p[tC] + (q[tC] - p[tC]) * (2/3. - t[tC]) * 6
        tD = t >= 2/3.
        x[tD] = p[tD]
        return x

    chroma = (s != 0)
    r[chroma] = channel(h + 1./3.)[chroma]
    g[chroma] = channel(h)[chroma]
    b[chroma] = channel(h - 1./3.)[chroma]

    return r, g, b

r2d, g2d, b2d = hsl_to_rgb(hue2d, sat2d, mag2d)
image_rgb = np.stack((r2d, g2d, b2d), axis=2)
plot.colorgram(image_rgb, x=step_rate)
